In [2]:
import pandas as pd
import numpy as np
import geopy.distance

## Cleaning Monthly Data

In [3]:
df=pd.read_csv('./ecobicidata/2021-07.csv')

In [4]:
def transform_df(ene=df):
    ene["full_date_retiro"] = pd.to_datetime(ene["Fecha_Retiro"] + " " + ene["Hora_Retiro"], format="%d/%m/%Y %H:%M:%S").copy()
    ene["full_date_aribo"] = pd.to_datetime(ene["Fecha Arribo"] + " " + ene["Hora_Arribo"], format="%d/%m/%Y %H:%M:%S").copy()
    ene["Mes"] = ene["full_date_retiro"].dt.month
    ene["Hora"] = ene["full_date_retiro"].dt.hour
    ene["time_delta"] = round((ene["full_date_aribo"]  - ene["full_date_retiro"]) / np.timedelta64(1,"m"),2)
    ene["Ciclo_Estacion_Retiro"]= ene[["Ciclo_Estacion_Retiro"]].astype(str)
    ene["Ciclo_Estacion_Retiro"] = [i[:-2] for i in ene["Ciclo_Estacion_Retiro"]]
    ene["Bici"]= ene[["Bici"]].astype(str)
    ene["Bici"] = [i[:-2] for i in ene["Bici"]]
    ene["viaje"] = ene["Ciclo_Estacion_Retiro"].astype(str)+"-"+ene["Ciclo_EstacionArribo"].astype(str)
    return ene.iloc[:,[0,1,2,3,6,9,10,11,12,13,14]]

In [5]:
ene = transform_df(ene=df)

## Cleaning Station Data

In [6]:
estaciones = pd.read_csv("./ecobicidata/estaciones-de-ecobici.csv")[["id","name","districtcode","districtname","location_lat","location_lon","stationtype","punto_geo"]]
estaciones["Ciclo_Estacion_Retiro"] = estaciones["id"].astype(str).copy()
estaciones["Ciclo_EstacionArribo"] = estaciones["id"].copy()
estaciones_retiro = estaciones.iloc[:,[-2,1,2,3,4,5,6,7]].rename(columns={"name":"name_retiro","districtcode":"districtcode_retiro","districtname":"districtname_retiro","location_lat":"location_lat_retiro","location_lon":"location_lon_retiro","stationtype":"stationtype_retiro","punto_geo":"punto_geo_retiro"}).copy().iloc[:,[0,1,4,5,7]]
estaciones_arribo = estaciones.iloc[:,[-1,1,2,3,4,5,6,7]].rename(columns={"name":"name_arribo","districtcode":"districtcode_arribo","districtname":"districtname_arribo","location_lat":"location_lat_arribo","location_lon":"location_lon_arribo","stationtype":"stationtype_arribo","punto_geo":"punto_geo_arribo"}).copy().iloc[:,[0,1,4,5,7]]

In [7]:
first = ene.merge(estaciones_retiro, on="Ciclo_Estacion_Retiro", how="left").merge(estaciones_arribo, on="Ciclo_EstacionArribo", how="left")

In [33]:
location_lat_retiro = first["location_lat_retiro"].fillna('19.412182').to_list()
location_lon_retiro = first["location_lon_retiro"].fillna('19.412182').to_list()
location_lat_arribo = first["location_lat_arribo"].fillna('19.412182').to_list()
location_lon_arribo = first["location_lon_arribo"].fillna('19.412182').to_list()

distances = pd.DataFrame({"location_dist":[geopy.distance.distance((location_lat_retiro[i],location_lon_retiro[i]), (location_lat_arribo[i],location_lon_arribo[i])).km for i in range(len(location_lon_arribo))]})

#location_list=[]
# for i in range(20):
#     coords_1=(location_lat_retiro[i],location_lon_retiro[i])
#     coords_2=(location_lat_arribo[i],location_lon_arribo[i])
#     location_list.append(geopy.distance.distance(coords_1, coords_2).km)

# pd.DataFrame({"location_dist":location_list})

# coords_1 = (19.412182, -99.158443)
# coords_2 = (19.40472, 19.40472)

# print(geopy.distance.distance(coords_1, coords_2).km)

In [28]:
test_head = first.head(20).copy()


20


In [34]:
l = pd.concat([first, distances], axis=1, join="inner").to_csv('./ecobicidata/2021_jul.csv')